In [1]:
%matplotlib inline
import os
import sys
import pandas as pd

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.features.build_features import load_data
from src.model.train import train_test

# Preface

This dataset involves predicting what category scientific papers fall into. There is a network of papers as well as the words used in the title.

## Load CORA Data

In [3]:
A, X, y, idx_train, idx_val, idx_test = load_data("../data/twitch/", "musae_ENGB_", 1000, 2000, 3000)

Loading musae_ENGB_ dataset...
Graph Info:
 Name: G
Type: Graph
Number of nodes: 7126
Number of edges: 35324
Average degree:   9.9141


/home/s3patil/DSC 180/DSC180-Replication-Project/src/model/utils.py:13: UserWarning: torch.eig is deprecated in favor of torch.linalg.eig and will be removed in a future PyTorch release.
torch.linalg.eig returns complex tensors of dtype cfloat or cdouble rather than real tensors mimicking complex tensors.
L, _ = torch.eig(A)
should be replaced with
L_complex = torch.linalg.eigvals(A)
and
L, V = torch.eig(A, eigenvectors=True)
should be replaced with
L_complex, V_complex = torch.linalg.eig(A) (Triggered internally at  /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:2897.)
  evals, evecs = torch.eig (m, eigenvectors = True)  # get eigendecomposition


Shape of A:  torch.Size([7126, 7126])

Shape of X:  torch.Size([7126, 3])

Adjacency Matrix (A):
 tensor([[0.1250, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0500, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.2500, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0476, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.3333]])

Node Features Matrix (X):
 tensor([[-0.0937, -0.0620, -0.2387],
        [ 0.1469, -0.0640, -0.2387],
        [-1.5768, -0.0495, -0.2387],
        ...,
        [-0.1517, -0.0639, -0.2387],
        [ 0.7370, -0.0631, -0.2387],
        [ 0.3846, -0.0640, -0.2387]])


# FCN (features only)

In [4]:
train_test(A, X, y, idx_train, idx_val, idx_test,
    no_cuda = False,
    seed = 40,
    epochs = 200,
    learning_rate = 0.0001,
    weight_decay = 5e-4,
    hidden_units = 256,
    dropout = 0.5,
    type = "FCN")

Epoch: 0010 loss_train: 0.7175 acc_train: 0.5000 loss_val: 0.7030 acc_val: 0.5045 time: 0.0039s
Epoch: 0020 loss_train: 0.7094 acc_train: 0.4930 loss_val: 0.6988 acc_val: 0.5055 time: 0.0035s
Epoch: 0030 loss_train: 0.6974 acc_train: 0.5360 loss_val: 0.6964 acc_val: 0.5120 time: 0.0033s
Epoch: 0040 loss_train: 0.7062 acc_train: 0.5270 loss_val: 0.6947 acc_val: 0.5185 time: 0.0031s
Epoch: 0050 loss_train: 0.7014 acc_train: 0.5280 loss_val: 0.6935 acc_val: 0.5260 time: 0.0032s
Epoch: 0060 loss_train: 0.7000 acc_train: 0.5290 loss_val: 0.6925 acc_val: 0.5365 time: 0.0051s
Epoch: 0070 loss_train: 0.7043 acc_train: 0.5260 loss_val: 0.6919 acc_val: 0.5415 time: 0.0046s
Epoch: 0080 loss_train: 0.7043 acc_train: 0.5140 loss_val: 0.6914 acc_val: 0.5440 time: 0.0037s
Epoch: 0090 loss_train: 0.7029 acc_train: 0.5050 loss_val: 0.6911 acc_val: 0.5440 time: 0.0041s
Epoch: 0100 loss_train: 0.6928 acc_train: 0.5410 loss_val: 0.6908 acc_val: 0.5440 time: 0.0038s
Epoch: 0110 loss_train: 0.6952 acc_train

# FCN (features and ad-hoc graph variables)

In [5]:
A1, X1, y1, idx_train, idx_val, idx_test = load_data("../data/twitch/", "musae_ENGB_", 1000, 2000, 3000, True)

Loading musae_ENGB_ dataset...
Graph Info:
 Name: G
Type: Graph
Number of nodes: 7126
Number of edges: 35324
Average degree:   9.9141
Shape of A:  torch.Size([7126, 7126])

Shape of X:  torch.Size([7126, 5])

Adjacency Matrix (A):
 tensor([[0.1250, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0500, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.2500, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0476, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.3333]])

Node Features Matrix (X):
 tensor([[-0.0937, -0.0620, -0.2387, -0.1313,  0.7731],
        [ 0.1469, -0.0640, -0.2387,  0.4095,  1.5530],
        [-1.5768, -0.0495, -0.2387, -0.0412,  0.2745],
        ...,
        [-0.1517, -0.0639, -0.2387, -0.3116, -0.5563],
        [ 0.7370, -0.0631, -0.2387,  0.4546,  0.9533],
        [ 0.3846, -0.0640, -0.2387, -0.3567, -0.9963]]

In [6]:
train_test(A1, X1, y1, idx_train, idx_val, idx_test,
    no_cuda = False,
    seed = 40,
    epochs = 200,
    learning_rate = 0.0001,
    weight_decay = 5e-4,
    hidden_units = 256,
    dropout = 0.5,
    type = "FCN")

Epoch: 0010 loss_train: 0.7009 acc_train: 0.5470 loss_val: 0.6907 acc_val: 0.5460 time: 0.0030s
Epoch: 0020 loss_train: 0.6965 acc_train: 0.5350 loss_val: 0.6890 acc_val: 0.5450 time: 0.0030s
Epoch: 0030 loss_train: 0.6983 acc_train: 0.5230 loss_val: 0.6881 acc_val: 0.5470 time: 0.0030s
Epoch: 0040 loss_train: 0.6939 acc_train: 0.5580 loss_val: 0.6875 acc_val: 0.5430 time: 0.0028s
Epoch: 0050 loss_train: 0.6963 acc_train: 0.5240 loss_val: 0.6872 acc_val: 0.5465 time: 0.0030s
Epoch: 0060 loss_train: 0.6926 acc_train: 0.5410 loss_val: 0.6869 acc_val: 0.5520 time: 0.0028s
Epoch: 0070 loss_train: 0.6911 acc_train: 0.5490 loss_val: 0.6866 acc_val: 0.5530 time: 0.0028s
Epoch: 0080 loss_train: 0.6855 acc_train: 0.5370 loss_val: 0.6864 acc_val: 0.5500 time: 0.0030s
Epoch: 0090 loss_train: 0.6959 acc_train: 0.5550 loss_val: 0.6862 acc_val: 0.5505 time: 0.0029s
Epoch: 0100 loss_train: 0.6902 acc_train: 0.5450 loss_val: 0.6860 acc_val: 0.5480 time: 0.0028s
Epoch: 0110 loss_train: 0.6841 acc_train

# node2vec (graph only)

node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)

 Embed nodes

mod = node2vec.fit(window=10, min_count=1, batch_words=4)

Results 

Epoch: 0190 loss_train: 0.6387 acc_train: 0.6500 loss_val: 0.7089 acc_val: 0.5180 time: 0.0031s

Epoch: 0200 loss_train: 0.6372 acc_train: 0.6580 loss_val: 0.7102 acc_val: 0.5220 time: 0.0030s

Optimization Finished!
Total time elapsed: 0.6735s
Test set results: loss= 0.7037 accuracy= 0.5180

# node2vec (graph and features)

Epoch: 0190 loss_train: 0.6347 acc_train: 0.6420 loss_val: 0.7088 acc_val: 0.5095 time: 0.0045s

Epoch: 0200 loss_train: 0.6377 acc_train: 0.6420 loss_val: 0.7098 acc_val: 0.5095 time: 0.0030s

Optimization Finished!
Total time elapsed: 0.6197s
Test set results: loss= 0.7161 accuracy= 0.5093

# GCN (nodes and features)

In [8]:
train_test(A, X, y, idx_train, idx_val, idx_test,
    no_cuda = False,
    seed = 40,
    epochs = 500,
    learning_rate = 0.0001,
    weight_decay = 5e-4,
    hidden_units = 256,
    dropout = 0.5,
    type = "GCN")

Epoch: 0010 loss_train: 0.6941 acc_train: 0.4540 loss_val: 0.6938 acc_val: 0.4560 time: 0.0495s
Epoch: 0020 loss_train: 0.6920 acc_train: 0.5460 loss_val: 0.6919 acc_val: 0.5440 time: 0.0479s
Epoch: 0030 loss_train: 0.6905 acc_train: 0.5460 loss_val: 0.6905 acc_val: 0.5440 time: 0.0488s
Epoch: 0040 loss_train: 0.6894 acc_train: 0.5460 loss_val: 0.6896 acc_val: 0.5440 time: 0.0490s
Epoch: 0050 loss_train: 0.6890 acc_train: 0.5460 loss_val: 0.6892 acc_val: 0.5440 time: 0.0486s
Epoch: 0060 loss_train: 0.6888 acc_train: 0.5460 loss_val: 0.6891 acc_val: 0.5440 time: 0.0494s
Epoch: 0070 loss_train: 0.6888 acc_train: 0.5460 loss_val: 0.6891 acc_val: 0.5440 time: 0.0486s
Epoch: 0080 loss_train: 0.6888 acc_train: 0.5460 loss_val: 0.6891 acc_val: 0.5440 time: 0.0478s
Epoch: 0090 loss_train: 0.6888 acc_train: 0.5460 loss_val: 0.6891 acc_val: 0.5440 time: 0.0486s
Epoch: 0100 loss_train: 0.6888 acc_train: 0.5460 loss_val: 0.6891 acc_val: 0.5440 time: 0.0482s
Epoch: 0110 loss_train: 0.6888 acc_train

# Discussion Questions

This node classification task is transductive as not all streamers are classified as Mature or not. Streamers aren't required to say if they use explicit language or not so it would make sense to look at the fixed network to figure out which streamers are mature are not using node classification.

The Train-Test split can be done on the feature level then so that the full network can still be inputted into the model since there are essentially unlabled gaps in the network that we would want to classify.

*Summarize how each ML approach handles inductive graph learning (adding new nodes and edges at test-time). What computation has to occur at test-time?*

FCN(only features): This does fine with inductive graph learning because there is no link between the data points.

FCN (features and ad-hoc graph variables): This also does fine since there are features to carry predictions even if new data points don't have ad-hoc summarizations. I think at test time this graph input has to be imputed.

All graph inputted approaches: Need to use something like GraphSAGE that generates embeddings by "sampling and aggregating features from a node’s local neighborhood"